<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/parity_qnn_noisy_vmap_Fallback_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane torch tqdm

In [ ]:
import pennylane as qml
from pennylane import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import time


def generate_parity_data(n_qubits):
    """
    Generate all 2^n binary strings of length n and their parity labels.
    Returns:
        X (torch.Tensor): shape (2^n, n)
        y (torch.Tensor): shape (2^n, 1)
    """
    X = np.array([list(map(int, format(i, f"0{n_qubits}b")))
                  for i in range(2 ** n_qubits)])
    y = np.mod(np.sum(X, axis=1), 2)
    return (
        torch.tensor(X, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    )


def noisy_ansatz(params, x=None, noise_prob=0.01):
    """
    Variational ansatz with data re-uploading and depolarizing noise.
    Args:
        params (array): shape (layers, wires, 3)
        x (array): binary input of length wires
        noise_prob (float): probability for depolarizing channel
    """
    n_layers, n_wires, _ = params.shape

    for i in range(n_wires):
        qml.RX(np.pi * x[i], wires=i)

    for layer in range(n_layers):
        for wire in range(n_wires):
            qml.Rot(*params[layer, wire], wires=wire)
            qml.DepolarizingChannel(noise_prob, wires=wire)

        for wire in range(n_wires - 1):
            qml.CNOT(wires=[wire, wire + 1])
        qml.CNOT(wires=[n_wires - 1, 0])


class HybridParityModel(nn.Module):
    """
    Hybrid QNN for n-bit parity.
    Quantum returns <Z_i> for each wire.
    Classical head: MLP with dropout + sigmoid.
    """
    def __init__(self, n_qubits, n_layers, n_hidden, dropout_p=0.1):
        super().__init__()
        self.n_qubits = n_qubits

        # Use mixed backend to support noise channels
        dev = qml.device("default.mixed", wires=n_qubits)

        @qml.qnode(dev, interface="torch", diff_method="backprop")
        def circuit(inputs, q_params):
            noisy_ansatz(q_params, x=inputs, noise_prob=0.02)
            return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

        self.qnode = circuit
        self.q_params = nn.Parameter(0.1 * torch.randn(n_layers, n_qubits, 3))

        self.classical = nn.Sequential(
            nn.Linear(n_qubits, n_hidden),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(n_hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        q_out = []
        for xi in x:
            z_list = self.qnode(xi, self.q_params)
            z_tensor = torch.tensor(z_list, dtype=torch.float32)
            q_out.append(z_tensor)
        q_out = torch.stack(q_out)  # shape: (batch, n_qubits)
        return self.classical(q_out)


def train_model(n_qubits, n_layers=3, n_hidden=8,
                epochs=30, lr=0.01):
    """
    Train the hybrid model on all parity patterns.
    Prints epoch-by-epoch loss and accuracy.
    """
    X, y = generate_parity_data(n_qubits)
    model = HybridParityModel(n_qubits, n_layers, n_hidden)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()

        preds = model(X)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            acc = ((preds > 0.5).float() == y).float().mean()
        print(f"Epoch {epoch:2d} | Loss: {loss.item():.4f} | Acc: {acc:.3f}")

    return model


def benchmark_inference(model, X):
    """
    Time per-sample QNode calls vs. PennyLane vmap (if available).
    """
    # Per-sample timing
    start = time.time()
    for xi in X:
        _ = model.qnode(xi, model.q_params)
    t_single = time.time() - start

    print(f"Per-sample QNode time: {t_single:.4f}s")

    # Try to import vmap
    try:
        from pennylane.transforms import vmap
    except ImportError:
        vmap = None

    if vmap:
        vmap_qnode = vmap(model.qnode, in_dims=(0, None))
        start = time.time()
        _ = vmap_qnode(X, model.q_params)
        t_vmap = time.time() - start
        print(f"vmap batched QNode time: {t_vmap:.4f}s")
    else:
        print("PennyLane vmap not available. Skipping batched timing.")


def mc_dropout_predict(model, X, mc_runs=50):
    """
    MC-Dropout uncertainty estimation.
    Returns mean and std of predictions.
    """
    model.eval()
    model.classical.train()

    preds = []
    for _ in range(mc_runs):
        preds.append(model(X))
    preds = torch.stack(preds)  # (mc_runs, batch, 1)
    mean = preds.mean(dim=0)
    std = preds.std(dim=0)

    model.classical.eval()
    return mean, std


if __name__ == "__main__":
    # Experiment: 6-qubit parity
    N = 6
    model_6q = train_model(N, n_layers=4, n_hidden=16,
                           epochs=30, lr=0.005)

    X6, _ = generate_parity_data(N)
    benchmark_inference(model_6q, X6)

    mean_preds, std_preds = mc_dropout_predict(model_6q, X6, mc_runs=100)
    print("Uncertainties for first 5 samples:")
    print(std_preds[:5].squeeze())